In [12]:
# This code sets up display options, imports, etc.
!pip install matplotlib_inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib_inline.backend_inline import set_matplotlib_formats

import plotly.express as px
import plotly.io as pio

# set up plotting defaults
%matplotlib inline
set_matplotlib_formats("svg")
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (4, 3)
pio.templates.default = "simple_white"

# display options for numpy and pandas
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option("display.max_rows", 7)
pd.set_option("display.max_columns", 8)
pd.set_option("display.precision", 2)

In [13]:
def display_df(
    df, rows=pd.options.display.max_rows, cols=pd.options.display.max_columns
):
    """Displays n rows and cols from df"""
    with pd.option_context("display.max_rows", rows, "display.max_columns", cols):
        display(df)

# Lab 4: Case Study on Air Quality

**Data Science Bootcamp with Python, Pandas, and Plotly**

Mar 14, 2023

## Part 1: Cleaning a US Sensor's Data

In [14]:
def data(csv):
    return f'https://github.com/DS-100/textbook/raw/master/content/datasets/purpleair_study/{csv}'

data('aqs_06-067-0010.csv')

In [15]:
aqs = pd.read_csv(data('aqs_06-067-0010.csv'))
aqs.head(2)

In [8]:
display_df(aqs.iloc[0].to_frame(), rows=31)

Data dictionary:

https://aqs.epa.gov/aqsweb/airdata/FileFormats.html#_hourly_data_files

### What should we check about the data quality?

https://learningds.org/ch/09/wrangling_checks.html

### What's the Granularity?

In [17]:
...

In [18]:
one_date = (aqs.query('date_local == "2018-12-31"')
 [['date_local', 'pollutant_standard', 'event_type', 'arithmetic_mean']]
)
display_df(one_date, rows=12)

In [19]:
...

So, we can simply take the first PM2.5 measurement for each date.

In [20]:
...

In [21]:
...

This data cleaning step gives us the desired granularity of `aqs`:
every row in `aqs` represents a single date, with an average PM2.5
measurement for that date.

## Removing Unneeded Columns

We plan to match the PM2.5 measurements in the `aqs` dataframe with
the PurpleAir PM2.5 measurements for each date.
To simplify the data, we'll subset out the date and PM2.5 columns and rename the
PM2.5 column so that it's easier to understand.

In [22]:
...

In [23]:
...

## Checking the Validity of `date_local`

Let's take a closer look at the `date_local` column.
We can already see that there are gaps in dates where there are no
PM2.5 readings.

In [24]:
# The table is sorted by `date_local`, so we see that there are missing dates
# between 2018-05-20 and 2018-05-23, for example.
aqs

In [25]:
# Python strings are recorded as the `object` type in pandas
aqs.dtypes

In [26]:
aqs['date_local'].iloc[0]

In [27]:
...

In [27]:
...

In [32]:
...

In [29]:
aqs.dtypes

Now that the `date_local` contains timestamps, we can calculate how
many dates are missing. We'll find the number of days between the earliest and
latest date---this corresponds to the maximum number of measurements we could
have recorded.

In [35]:
date_range = aqs['date_local'].max() - aqs['date_local'].min()
date_range

In [36]:
# Subtracting timestamps give Timedelta objects, which have a few useful
# properties like:
date_range.days

In [39]:
print(f'We have {len(aqs)} / {date_range.days} measurements, '
      f'or {len(aqs) / date_range.days:.0%} of the dates possible.')

### Checking the Validity of `pm25`

In [30]:
px.scatter(aqs, x='date_local', y='pm25');

## End of Part 1!

## Part 2: Cleaning a PurpleAir Sensor's Data

In [33]:
pa_raw = pd.read_csv(data('purpleair_AMTS/AMTS_TESTING%20(outside)%20(38.568404%20-121.493163)%20Primary%20Real%20Time%2005_20_2018%2012_29_2019.csv'))
pa_raw

In [34]:
display_df(pa_raw.iloc[0].to_frame().reset_index(), rows=11)

There are two columns that
contain PM2.5 data: `PM2.5_CF1_ug/m3` and `PM2.5_ATM_ug/m3`.
We'll keep `PM2.5_CF1_ug/m3` since the original researchers found that feature to be
more effective.

In [35]:
def subset_and_rename_A(df):
    df = df[['created_at', 'PM2.5_CF1_ug/m3', 'Temperature_F', 'Humidity_%']]
    df.columns = ['timestamp', 'PM25cf1', 'TempF',
                  'RH'] # RH stands for Relative Humidity
    return df

pa = (pa_raw
      .pipe(subset_and_rename_A))
pa

### What's the Granularity?

In [8]:
pa.head(2)

In [36]:
...

In [38]:
...

Need to adjust for timezone!

In [39]:
# https://pvlib-python.readthedocs.io/en/stable/timetimezones.html
# The US/Pacific timezone corresponds to the timezone in California, and will
# automatically adjust for Daylight Saving Time.
pa.tz_convert('US/Pacific')

In [40]:
...

### Visualizing Timestamps

In [41]:
# The 'D' argument tells resample to aggregate timestamps into individual dates
(pa.resample('D')
 .size() # We can call aggregation methods just like we would with `groupby()`
)

In [48]:
per_day = (pa
 .resample('D')
 .size()
 .rename('# measurements')
 .reset_index()
)
per_day

In [49]:
px.line(per_day, x='timestamp', y='# measurements')

PurpleAir says that its sensors take a recording every 2 minutes. What's weird about this plot?

#### Why is the Sampling Rate Inconsistent?

In [71]:
# Passing a string into .loc will filter timestamps
...

In [19]:
pa.loc['2019-01-01 00:00']

In [18]:
...

In [72]:
...

In [73]:
per_day = (pa
 .resample('D')
 .size()
 .rename('# measurements')
 .reset_index()
)
per_day

In [74]:
px.line(per_day, x='timestamp', y='# measurements')

### You Try: What do we do About Missing Data?

## Part 3: Exploring the Sensor Pair's Data

What aspects of this data can we explore?

In [80]:
csv_file = 'data/cleaned_purpleair_aqs/Full24hrdataset.csv'
usecols = ['Date', 'ID', 'region', 'PM25FM', 'PM25cf1', 'TempC', 'RH', 'Dewpoint']

full_df = (pd.read_csv(data('cleaned_purpleair_aqs/Full24hrdataset.csv'),
                       usecols=usecols, parse_dates=['Date'])
           .dropna())
full_df.columns = ['date', 'id', 'region', 'pm25aqs', 'pm25pa', 'temp', 'rh', 'dew']
full_df

| Column  | Description                                                                                                                      |
|---------|----------------------------------------------------------------------------------------------------------------------------------|
| date    | Date of the observation                                                                                                          |
| id      | A unique label for a site, formatted as the US state abbreviation with a number. (We performed data cleaning for site ID `CA1`.) |
| region  | The name of the region, which corresponds to a group of sites. (The `CA1` site is located in the `West` region.)                 |
| pm25aqs | The PM2.5 measurement from the AQS sensor.                                                                                       |
| pm25pa  | The PM2.5 measurement from the PurpleAir sensor.                                                                                 |
| temp    | Temperature, in Celcius.                                                                                                         |
| rh      | Relative humidity, ranging from 0 to 100%.                                                                                       |
| dew     | The dew point. (Higher dew point means more moisture in the air.)                                                     |

In [87]:
df = (full_df
 .query('id == "CA1"')
 .melt(['date'], ['pm25aqs', 'pm25pa'], var_name='sensor', value_name='pm25')
 .sort_values('date')
)
px.line(df, x='date', y='pm25', color='sensor')

In [90]:
px.scatter(full_df, x='pm25aqs', y='pm25pa')

In [94]:
under_50 = full_df[full_df['pm25aqs'] < 50]
px.scatter(under_50, x='pm25aqs', y='pm25pa',
           trendline='lowess',
           trendline_color_override="orange")

In [96]:
full_df[['pm25aqs', 'pm25pa']].corr()

## Part 4: Modeling the Difference between US and PurpleAir

Don't worry about the code below, I'll explain as we go.

In [105]:
np.random.seed(42)

n = len(full_df)
test_n = 1000

# Shuffle the row labels
shuffled = np.random.choice(n, size=n, replace=False)

# Split the data
test  = full_df.iloc[shuffled[:test_n]]
train = full_df.iloc[shuffled[test_n:]]
train

In [103]:
def rmse(predictions):
    return np.sqrt(np.mean((test['pm25aqs'] - predictions)**2))

In [104]:
def model_results(models):
    results = [
        [model.__doc__, rmse(model(train)(test))]
        for model in models
    ]
    return (pd.DataFrame(results, columns=['model', 'rmse'])
            .set_index('model'))

### One-variable model (simple linear regression)

In [106]:
from sklearn.linear_model import LinearRegression

def model_1(train):
    '''f(x) = θ₀ + θ₁PA'''
    # Fit calibration model using sklearn
    X, y = train[['pm25aqs']], train['pm25pa']
    model = LinearRegression().fit(X, y)
    m, b = model.coef_[0], model.intercept_
    
    # Invert model
    theta_1 = 1 / m
    theta_0 = - b / m
    
    def predict(data):
        return theta_0 + theta_1 * data['pm25pa']
    return predict

In [107]:
model_results([model_1])

Under this model, $ \hat \theta_1 = 0.52 $ and $ \hat \theta_0 = 1.54 $, so our
fitted model predicts:

$$
\begin{aligned}
\text{PM}_{2.5} = 1.54 + 0.52 \text{PA}
\end{aligned}
$$

### Model with both PurpleAir and Relative Humidity

In [108]:
def model_2(train):
    '''f(x) = θ₀ + θ₁PA + θ₂RH'''
    # Fit calibration model using sklearn
    X, y = train[['pm25aqs', 'rh']], train['pm25pa']
    model = LinearRegression().fit(X, y)
    [m1, m2], b = model.coef_, model.intercept_
    
    # Invert to find parameters
    theta_0 = - b / m1
    theta_1 = 1 / m1
    theta_2 = - m2 / m1
    
    def predict(data):
        return theta_0 + theta_1 * data['pm25pa'] + theta_2 * data['rh']
        return (data['pm25pa'] - data['rh'] * m2 - b) / m1
    return predict

In [109]:
model_results([model_1, model_2])

After fitting the model, we have:
$ \hat \theta_0 = 5.77 $, $ \hat \theta_1 = 0.524 $, and
$ \hat \theta_2 = -0.0860 $
so our fitted model predicts:

$$
\begin{aligned}
\text{PM}_{2.5} = 5.77 + 0.524 \cdot \text{PA} - 0.086 \cdot \text{RH}
\end{aligned}
$$